In [1]:
import os
import pymongo

from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv("../env")
client = MongoClient(os.environ.get("MONGO_DB_CONNECTION"))
db = client.get_database("prismai")
collection = db.get_collection("transition_scores")

In [3]:
# db.drop_collection("transition_scores")

{'ok': 1.0}

In [ ]:
collection.count_documents({"_split": {"$exists": False}})

In [ ]:
collection.count_documents({"_split": ".0"})

In [ ]:
collection.count_documents({"_split": ".0.0"})

In [ ]:
collection.count_documents({"_split": ".0.0.0"})

In [ ]:
collection.count_documents({"_split": ".0.0.0.0"})